In [5]:

#creating cnn model

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# weight_variable: -0.2~0.2
#將shape的每個element設為從常態分佈隨機取一個數, 大於2個標準差的數會drop, re-pick
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#bias = 0.1
#Creates a constant tensor and initialize it to 0.1
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)



#2-D CNN
#stride(位移量) = [1, stride,stride, 1]
def conv(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'SAME')

def relu(data):
    return tf.nn.relu(data)


#x(input) = [batch, height, width, channels]
#ksize(窗口大小) = [1, height, width, 1]
#stride = [1, stride,stride, 1]
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')


def fully_connected(data, shape, weight, bias):
        flatten = tf.reshape(data, shape)
        #NOT CNN!!
        fc = tf.nn.relu(tf.matmul(flatten, weight) + bias)
        return fc


def drop_out(h_fc, keep_prob):
    #dropout
    #避免過度擬合(over fitting)
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)
    return h_fc_drop


def cross_entropy(y_predict, label):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels=label))
    #優化器: 使cross entropy最小化
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    #預測準確度
    #argmax: 得到Tensor中最大值的index
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(label,1))
    #神經網路準確率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return train_step, accuracy




def softmax(W_fc, b_fc, h_fc_drop):
    #softmax
    #轉為0~1的機率
    y_predict = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc)+b_fc)
    return y_predict





/Users/user20/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
#the whole cnn model

# model:
# Input   ->   Conv   -> ReLU ->   Pool   ->   Conv   -> ReLU ->  Pool  -> Fully Conneted
# 
# size:(row * column * features)
# 28*28*1 -> 28*28*32 -> .... -> 14*14*32 -> 14*14*64 -> .... -> 7*7*64 -> 1024




class cnn_model_2_layer:
    
    def __init__(self):
        
        self.img_placeholder = tf.placeholder(tf.float32, [None, 784])
        self.label_placeholder = tf.placeholder(tf.float32, [None, 10])
        #input dimantion: -1(不考慮維度), size: 28*28, channel:1
        self.input_img = tf.reshape(self.img_placeholder, [-1, 28, 28, 1])
        
        
        self.W_conv1 = weight_variable([5, 5, 1, 32]) 
        self.b_conv1 = bias_variable([32])
        self.W_conv2 = weight_variable([5, 5, 32, 64]) 
        self.b_conv2 = bias_variable([64])
    
        self.W_fc1 = weight_variable([7 * 7 * 64, 1024])
        self.b_fc1 = bias_variable([1024])
        self.W_fc2 = weight_variable([1024, 10])
        self.b_fc2 = bias_variable([10])
        
        
        self.fc_shape = [-1, 7 * 7 * 64]
        self.keep_prob =  tf.placeholder(tf.float32)

    def set_value(self, W_conv1, W_conv2, b_conv1, b_conv2, W_fc1, W_fc2, b_fc1, b_fc2):
        self.W_conv1 = W_conv1
        self.b_conv1 = b_conv1
        self.W_conv2 = W_conv2
        self.b_conv2 = b_conv2
        self.W_fc1 = W_fc1
        self.b_fc1 = b_fc1
        self.W_fc2 = W_fc2
        self.b_fc2 = b_fc2
        return
    
    

    
    def run_cnn(self):
        conv1 = conv(self.input_img, self.W_conv1)
        relu1 = relu(conv1 + self.b_conv1)
        pool1 = max_pool_2x2(relu1)
        
        conv2 = conv(pool1, self.W_conv2)
        relu2 = relu(conv2 + self.b_conv2)
        pool2 = max_pool_2x2(relu2)
        
        #fully connect
        fc = fully_connected(pool2, self.fc_shape, self.W_fc1, self.b_fc1)
        
        #dropout
        fc_drop = drop_out(fc, self.keep_prob)
        
        y_predict = softmax(self.W_fc2, self.b_fc2, fc_drop)
        
        #optimizer
        train_step, accuracy = cross_entropy(y_predict, self.label_placeholder)
        
        return train_step, accuracy
        
        '''
        h_pool1 = cnn_pooling(self.W_conv1, self.b_conv1, self.input_img)
        h_pool2 = cnn_pooling(self.W_conv2, self.b_conv2, h_pool1)
        
        h_fc1 = full_connect(self.W_fc1, self.b_fc1, h_pool2)
        h_fc_drop1 = drop_out(h_fc1, self.keep_prob)
        
        y_predict = softmax(self.W_fc2, self.b_fc2, h_fc_drop1)
        return y_predict
        '''
    
    def details(self):
        print("{:30}:{}".format("convolution weight 1", self.W_conv1))
        print("{:30}:{}".format("convolution bias 1", self.b_conv1))
        print("{:30}:{}".format("convolution weight 2", self.W_conv2))
        print("{:30}:{}".format("convolution bias 2", self.b_conv2))
        
        print("{:30}:{}".format("full connected weight 1", self.W_fc1))
        print("{:30}:{}".format("full connected bias 1", self.b_fc1))
        print("{:30}:{}".format("full connected weight 2", self.W_fc2))
        print("{:30}:{}".format("full connected bias 2", self.b_fc2))
        return
        



In [7]:
# model:
# model:
# input layer: [.,784]
#layer: Conv1 -> ReLu -> MaxPool: [.,14,14,32]
#layer: FC -> ReLu: [.,1024]
#output layer: FC -> ReLu: [.,10]



class cnn_model_1_layer:
    
    def __init__(self):
        
        self.img_placeholder = tf.placeholder(tf.float32, [None, 784])
        self.label_placeholder = tf.placeholder(tf.float32, [None, 10])
        #input dimantion: -1(不考慮維度), size: 28*28, channel:1
        self.input_img = tf.reshape(self.img_placeholder, [-1, 28, 28, 1])
        
        
        self.W_conv1 = weight_variable([5, 5, 1, 32]) 
        self.b_conv1 = bias_variable([32])
    
        self.W_fc1 = weight_variable([14 * 14 * 32, 1024])
        self.b_fc1 = bias_variable([1024])
        self.W_fc2 = weight_variable([1024, 10])
        self.b_fc2 = bias_variable([10])
        
        
        self.fc_shape = [-1, 14 * 14 * 32]
        self.keep_prob =  tf.placeholder(tf.float32)

    
    def run_cnn(self):
        conv1 = conv(self.input_img, self.W_conv1)
        relu1 = relu(conv1 + self.b_conv1)
        pool1 = max_pool_2x2(relu1)
        
        
        fc = fully_connected(pool1, self.fc_shape, self.W_fc1, self.b_fc1)
        
        fc_drop = drop_out(fc, self.keep_prob)
        
        y_predict = softmax(self.W_fc2, self.b_fc2, fc_drop)
        
        train_step, accuracy = cross_entropy(y_predict, self.label_placeholder)
        
        return train_step, accuracy
    
    

In [1]:
#model reference from: https://www.kaggle.com/raoulma/mnist-image-class-tensorflow-cnn-99-51-test-acc
#
# model:
# input layer: [.,784]
#layer: Conv1 -> ReLu -> MaxPool: [.,14,14,36]
#layer: Conv2 -> ReLu -> MaxPool: [.,7,7,36]
#layer: Conv3 -> ReLu -> MaxPool: [.,4,4,36]
#layer: FC -> ReLu: [.,576]
#output layer: FC -> ReLu: [.,10]
#
#
#
#mini batch size: 50
#keep_prob = 0.33



class cnn_model_3_layer:
    
    def __init__(self):
        
        self.img_placeholder = tf.placeholder(tf.float32, [None, 784])
        self.label_placeholder = tf.placeholder(tf.float32, [None, 10])
        
        self.input_img = tf.reshape(self.img_placeholder, [-1, 28, 28, 1])
        
        #convolution 1 filter: 3x3
        self.W_conv1 = weight_variable([3, 3, 1, 36])
        #convolution 2 filter: 3x3
        self.W_conv2 = weight_variable([3, 3, 36, 36])
        #convolution 3 filter: 3x3
        self.W_conv3 = weight_variable([3, 3, 36, 36])
        
        self.b_conv1 = bias_variable([36])
        self.b_conv2 = bias_variable([36])
        self.b_conv3 = bias_variable([36])
    
        
        self.W_fc1 = weight_variable([4 * 4 * 36, 576])
        self.W_fc2 = weight_variable([576, 10])
        self.b_fc1 = bias_variable([576])
        self.b_fc2 = bias_variable([10])
        
        
        self.fc_shape = [-1, 4 * 4 * 36]
        self.keep_prob = tf.placeholder(tf.float32)
        

    
    def run_cnn(self):
        conv1 = conv(self.input_img, self.W_conv1)
        relu1 = relu(conv1 + self.b_conv1)
        pool1 = max_pool_2x2(relu1)
        
        
        
        conv2 = conv(pool1, self.W_conv2)
        relu2 = relu(conv2 + self.b_conv2)
        pool2 = max_pool_2x2(relu2)
        
        conv3 = conv(pool2, self.W_conv3)
        relu3 = relu(conv3 + self.b_conv3)
        pool3 = max_pool_2x2(relu3)
        
        
        fc = fully_connected(pool3, self.fc_shape, self.W_fc1, self.b_fc1)
        
        fc_drop = drop_out(fc, self.keep_prob)
        
        y_predict = softmax(self.W_fc2, self.b_fc2, fc_drop)
        
        ztrain_step, accuracy = cross_entropy(y_predict, self.label_placeholder)
        
        return train_step, accuracy
